# Tutorial Document
This document details the use of the Multi Proxy library. The goal is a python package that allows users to interact with Aquila and DMRG data seemlessly and prevent repetitive function writing. 

### Getting Data from DMRG files
If you have data in text files with the form

state : count

state : count

.
.
.

Then the function below allows you to easily obtain and return the data as a dictionary and additionally the total count of the values (equals 1 if the values are probabilities) using the file path as input.

This function also has a built in progress manager so you can estimate how long the data takes to load in. This is helpful for larger datasets. To track progress use the "show_progress" flag and set it equal to True. This flag is always the last parameter passed into the function call. It is set to False by default. 

Note: 

You must set the flag in the function call itself like shown below if the function has multiple arguments before show_progress. You can try for yourself but if you just do show_progress = True on a separate line and pass that into the function it will not show progress. This is because parse_file also takes in an additional two arguments, one of which (sample_size) can be represented as a boolean. Thus if you just pass file_path and show_progress with show_progress defined on a previous line, it will read show_progress as the sample size input and it will not show progress. 

In [2]:
from Processor import parse_file

file_path = "../EntanglementCalculation/1_billion_shots/Rba_2.0/16_rungs.txt"

# Process without sampling
processed_data, total_count = parse_file(file_path, show_progress = False)

### Most probable states
This is great but it's hard to confirm for very large datasets. To confirm the result we might just be interested in the 10 most probable states. The function below allows us to print out the n most probable states

In [3]:
from Processor import print_most_probable_data

n = 10
print_most_probable_data(processed_data, n)

Most probable 10 bit strings:
 1.  Bit string: 11000110011001100110011001100011, Probability: 0.00534007
 2.  Bit string: 11001001100110011001100110010011, Probability: 0.00533582
 3.  Bit string: 11001100100110011001100110010011, Probability: 0.00224794
 4.  Bit string: 11000110011001100110011000110011, Probability: 0.00224652
 5.  Bit string: 11001001100110011001100100110011, Probability: 0.00224516
 6.  Bit string: 11001100011001100110011001100011, Probability: 0.00224463
 7.  Bit string: 11001001100110011000100110010011, Probability: 0.00179124
 8.  Bit string: 11001001100100011001100110010011, Probability: 0.00178862
 9.  Bit string: 11000110011001100100011001100011, Probability: 0.00178772
10.  Bit string: 11000110011001100010011001100011, Probability: 0.00178722


### Sampling 
If I want to take a random sample of the larger data into a smaller amount I can do so using the sample_data function

In [4]:
from Processor import sample_data
from Processor import print_most_probable_data

sample_size = 1000
sampled_data = sample_data(processed_data, total_count, sample_size, show_progress= True)
print_most_probable_data(sampled_data, 10)

Starting: Sampling data...
Task: Sampling data | Progress: 100.00% | Elapsed: 0.19s | Remaining: 0.00s     
Completed: Sampling data. Elapsed time: 0.19 seconds.
Most probable 10 bit strings:
 1.  Bit string: 11001001100110011001100110010011, Probability: 0.00600000
 2.  Bit string: 11001100011001100110011001000011, Probability: 0.00400000
 3.  Bit string: 11000110001001100110011001100011, Probability: 0.00400000
 4.  Bit string: 11001001100110010001100110010011, Probability: 0.00400000
 5.  Bit string: 11000110001001100110010010010011, Probability: 0.00300000
 6.  Bit string: 11000110011001100100011001100011, Probability: 0.00300000
 7.  Bit string: 11000110011001001100011001100011, Probability: 0.00300000
 8.  Bit string: 11001100100110011001100110010011, Probability: 0.00300000
 9.  Bit string: 11001001100110011001000110010011, Probability: 0.00300000
10.  Bit string: 11001001100110010010000110010011, Probability: 0.00300000


### Sample as you parse
It's actually more efficient to sample as we parse the set. So if you know before hand you only care about 100 randomly sample states it makes no sense to parse the whole thing and then sample it down to 100. Instead as you are parsing you will only grab 100 states. We can do this using the parse_file function by adding the sample_size parameter

In [1]:
from Processor import parse_file

file_path = "../EntanglementCalculation/1_billion_shots/Rba_2.0/14_rungs.txt"

# Process with sampling

sample_size = 1000
processed_data, total_count = parse_file(file_path, sample_size)

### Error handling
The Aquila device has a readout error rate of 0.08 for the excited state and 0.01 for the ground state. We can simulate this error on our data using the following function

NOTE: The current function assumes the default values of 0.08 and 0.01. These are taken as parameters so if future error rates change then we can accurately model those as well. Thus, you do not technically need to pass ground_rate and excited_rate into the function. Although it is good practice.

In [4]:
from Processor import introduce_error_data
from Processor import print_most_probable_data

ground_rate = 0.01
excited_rate = 0.08

error_data = introduce_error_data(sampled_data, total_count, ground_rate, excited_rate)
print_most_probable_data(error_data, 10)

Introducing errors to the data...
Most probable 10 bit strings:
 1.  Bit string: 0000000110010011000110010011, Probability: 0.00909091
 2.  Bit string: 1100110011000001001100010011, Probability: 0.00909091
 3.  Bit string: 1000110000010000100100100110, Probability: 0.00909091
 4.  Bit string: 1100100000101001100010000011, Probability: 0.00909091
 5.  Bit string: 0100000110011000100100010010, Probability: 0.00909091
 6.  Bit string: 0100100100110000100110010010, Probability: 0.00909091
 7.  Bit string: 0100100000010001000110010001, Probability: 0.00909091
 8.  Bit string: 1100110000000000000001000010, Probability: 0.00909091
 9.  Bit string: 1100010000110010010010010011, Probability: 0.00909091
10.  Bit string: 1000011001000100010010000010, Probability: 0.00909091


### Combining data
Say you have two data sets and you want to combine them. We can do this using the following funciton, however there are some rules. You can either combine two datasets of probabilities or two datasets of counts. You cannot combine a dataset of probabilities and a dataset of counts as this would make normalizing impossible. Additionally, if you combine two probabilities, the function will automatically normalize. If you combine two datasets of counts, the function will NOT normalize. This is so if users want to combine say 100 sets of count data, they can do so without a problem. They simply need to normalize afterwards. 

In [5]:
from Processor import combine_datasets

combined_data = combine_datasets(sampled_data, error_data)
print(combined_data)

{'1101010011000000100110000011': 0.004545454545454558, '1100110010011000100001001001': 0.0005000000000000014, '1100110011000001001100010011': 0.03654545454545465, '1100100011000100011001000010': 0.004545454545454558, '1100001001001001000010010001': 0.0005000000000000014, '0100100010010000000110010010': 0.012500000000000035, '1100011001100110000000100000': 0.012045454545454578, '1100010000100010010010010011': 0.03300000000000009, '0100000010010000110010010010': 0.0005000000000000014, '0100010000000011001100000011': 0.004545454545454558, '0100000110010011000010010011': 0.013500000000000038, '0100110010001100011000010010': 0.004545454545454558, '1100001101100100011001100010': 0.0005000000000000014, '0100100100110000100110010010': 0.03904545454545466, '1100110000011001100110011000': 0.0050454545454545596, '1000100010001100110001000010': 0.0005000000000000014, '1100011000010001000100000010': 0.004545454545454558, '1100010000010010000110010010': 0.0050454545454545596, '1100011001000000100011

### Saving to txt File
If we have a set of data we would like to save to a file we can do so using the following function

In [6]:
from Processor import save_data

file_path = "error_data.txt"
save_data(processed_data, file_path)

Starting: Saving data...
Task: Saving data | Progress: 100.00% | Elapsed: 0.91s | Remaining: 0.00s
Completed: Saving data. Elapsed time: 0.91 seconds.
